### Hate Speech Detection using PyTorch and Hugging Face Transformers
Hate speech detection task to determine if a piece of text contains hateful content.

Data: A twitter corpus study of US Elections 2020 on the basis of Offensive speech and Stance detection.

Data URL: https://www.ims.uni-stuttgart.de/forschung/ressourcen/korpora/stance-hof/

#### GPU Setup

In [1]:
import torch

ModuleNotFoundError: No module named 'torch'

In [2]:
print(torch.__file__)

NameError: name 'torch' is not defined

In [1]:
import torch

# If GPU is available
if torch.cuda.is_available():
    # PyTorch will use GPU
    device = torch.device("cuda")
    print("The GPU that is used: ", torch.cuda.get_device_name(0))

# If GPU is not available
else:
    # PyTorch will use CPU
    device = torch.device("cpu")
    print("No GPU available, CPU is used.")